In [1]:
import pandas as pd
import os
from pathlib import Path
import re

def load_trading_data(data_folder: str) -> dict[str, pd.DataFrame]:
    """
    Load trading data from CSV files into pandas DataFrames, agnostic to round and day.

    Args:
        data_folder (str): Path to the folder containing the CSV files

    Returns:
        Dict[str, pd.DataFrame]: Dictionary containing the following DataFrames:
            - 'prices': Price data for all days/rounds
            - 'trades': Trade data for all days/rounds
            - 'observations': Observation data for all days/rounds
    """
    data = {
        'prices': [],
        'trades': [],
        'observations': []
    }

    # Regex to match files and extract type, round, and day
    pattern = re.compile(r'^(prices|trades|observations)_round_(\d+)_day_(\d+)\.csv$')

    for file in Path(data_folder).iterdir():
        if file.is_file():
            match = pattern.match(file.name)
            if match:
                data_type, round_num, day = match.groups()
                sep = ';' if data_type in ['prices', 'trades'] else ','
                df = pd.read_csv(file, sep=sep)
                df['day'] = int(day)
                df['round'] = int(round_num)
                if 'timestamp' in df.columns:
                    df = df.sort_values('timestamp')
                data[data_type].append(df)

    # Concatenate all days' data
    result = {}
    for key, dfs in data.items():
        if dfs:  # Only add if we found any data
            result[key] = pd.concat(dfs, ignore_index=True)
            result[key].sort_values(['day', 'timestamp'], inplace=True)

    return result

def get_product_data(df: pd.DataFrame, product: str) -> pd.DataFrame:
    """
    Filter DataFrame for a specific product.

    Args:
        df (pd.DataFrame): DataFrame containing trading data
        product (str): Product name to filter for

    Returns:
        pd.DataFrame: Filtered DataFrame containing only data for the specified product
    """
    return df[df['product'] == product]

def get_day_data(df: pd.DataFrame, day: int) -> pd.DataFrame:
    """
    Filter DataFrame for a specific day.

    Args:
        df (pd.DataFrame): DataFrame containing trading data
        day (int): Day number to filter for

    Returns:
        pd.DataFrame: Filtered DataFrame containing only data for the specified day
    """
    return df[df['day'] == day]

def get_product_day_data(df: pd.DataFrame, product: str, day: int) -> pd.DataFrame:
    """
    Filter DataFrame for a specific product and day.

    Args:
        df (pd.DataFrame): DataFrame containing trading data
        product (str): Product name to filter for
        day (int): Day number to filter for

    Returns:
        pd.DataFrame: Filtered DataFrame containing only data for the specified product and day
    """
    return df[(df['product'] == product) & (df['day'] == day)]

def get_price_data(df: pd.DataFrame, product: str = None, day: int = None) -> pd.DataFrame:
    """
    Get price data with optional filtering by product and/or day.

    Args:
        df (pd.DataFrame): DataFrame containing price data
        product (str, optional): Product name to filter for
        day (int, optional): Day number to filter for

    Returns:
        pd.DataFrame: Filtered price data
    """
    result = df.copy()
    if product:
        result = result[result['product'] == product]
    if day:
        result = result[result['day'] == day]
    if 'timestamp' in result.columns:
        result = result.sort_values('timestamp')
    return result

def get_order_book_data(df: pd.DataFrame, product: str = None, day: int = None) -> pd.DataFrame:
    """
    Get order book data with optional filtering by product and/or day.

    Args:
        df (pd.DataFrame): DataFrame containing price data
        product (str, optional): Product name to filter for
        day (int, optional): Day number to filter for

    Returns:
        pd.DataFrame: Filtered order book data
    """
    result = df.copy()
    if product:
        result = result[result['product'] == product]
    if day:
        result = result[result['day'] == day]
    if 'timestamp' in result.columns:
        result = result.sort_values('timestamp')
    return result

def get_volume_data(df: pd.DataFrame, product: str = None, day: int = None) -> pd.DataFrame:
    """
    Get volume data with optional filtering by product and/or day.

    Args:
        df (pd.DataFrame): DataFrame containing trade data
        product (str, optional): Product name to filter for
        day (int, optional): Day number to filter for

    Returns:
        pd.DataFrame: Filtered volume data
    """
    result = df.copy()
    if product:
        result = result[result['product'] == product]
    if day:
        result = result[result['day'] == day]
    if 'timestamp' in result.columns:
        result = result.sort_values('timestamp')
    return result

def convert_timestamp(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(t=(df['day'] - 1) * 1_000_000 + df['timestamp']).drop(columns=['day', 'timestamp'])

In [2]:
trading_data = load_trading_data('round-5-island-data-bottle')

In [3]:
price_df = trading_data['prices']
price_df = convert_timestamp(price_df)
price_df

,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss,round,t
0,PICNIC_BASKET2,30094.0,11.0,30093.0,27.0,NaN,NaN,30099.0,1.0,30100.0,37.0,NaN,NaN,30096.5,0.0,5,1000000
1,PICNIC_BASKET1,58702.0,11.0,58701.0,27.0,NaN,NaN,58712.0,11.0,58713.0,27.0,NaN,NaN,58707.0,0.0,5,1000000
2,VOLCANIC_ROCK_VOUCHER_10500,8.0,11.0,NaN,NaN,NaN,NaN,9.0,11.0,NaN,NaN,NaN,NaN,8.5,0.0,5,1000000
3,VOLCANIC_ROCK_VOUCHER_10000,233.0,11.0,NaN,NaN,NaN,NaN,234.0,11.0,NaN,NaN,NaN,NaN,233.5,0.0,5,1000000
4,JAMS,6519.0,59.0,6518.0,196.0,NaN,NaN,6520.0,59.0,6521.0,196.0,NaN,NaN,6519.5,0.0,5,1000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,PICNIC_BASKET1,58434.0,1.0,58433.0,10.0,58432.0,30.0,58443.0,1.0,58444.0,10.0,58445.0,30.0,58438.5,0.0,5,3999900
449996,VOLCANIC_ROCK_VOUCHER_9750,356.0,11.0,NaN,NaN,NaN,NaN,357.0,11.0,NaN,NaN,NaN,NaN,356.5,0.0,5,3999900
449997,RAINFOREST_RESIN,9992.0,35.0,NaN,NaN,NaN,NaN,9999.0,1.0,10008.0,35.0,NaN,NaN,9995.5,0.0,5,3999900
449998,VOLCANIC_ROCK_VOUCHER_10000,122.0,11.0,NaN,NaN,NaN,NaN,123.0,11.0,NaN,NaN,NaN,NaN,122.5,0.0,5,3999900


In [4]:
observation_df = trading_data['observations']
observation_df = convert_timestamp(observation_df)
observation_df

,bidPrice,askPrice,transportFees,exportTariff,importTariff,sugarPrice,sunlightIndex,round,t
10000,657.0,658.5,1.4,10.5,-4.0,209.000000,65.00,5,1000000
10001,660.5,662.0,1.4,10.5,-4.0,209.299299,65.00,5,1000100
10002,664.0,665.5,1.4,10.5,-4.0,209.467113,65.00,5,1000200
10003,669.0,670.5,1.4,10.5,-4.0,209.525592,65.00,5,1000300
10004,673.5,675.0,1.4,10.5,-4.0,209.597333,65.00,5,1000400
...,...,...,...,...,...,...,...,...,...
9995,745.0,746.5,1.6,7.5,-4.0,206.028012,59.94,5,3999500
9996,741.0,742.5,1.6,7.5,-4.0,205.991170,59.95,5,3999600
9997,744.5,746.0,1.6,7.5,-4.0,206.053141,59.96,5,3999700
9998,744.0,745.5,1.6,7.5,-4.0,205.751795,59.98,5,3999800


In [5]:
counterparty_df = trading_data['trades']
counterparty_df = convert_timestamp(counterparty_df).drop(columns='round')
counterparty_df

,buyer,seller,symbol,currency,price,quantity,t
35229,Caesar,Paris,CROISSANTS,SEASHELLS,4265.0,8,1000000
35230,Charlie,Paris,RAINFOREST_RESIN,SEASHELLS,9999.0,1,1000100
35231,Paris,Caesar,CROISSANTS,SEASHELLS,4266.0,8,1000200
35232,Paris,Caesar,JAMS,SEASHELLS,6520.0,7,1000200
35233,Paris,Charlie,KELP,SEASHELLS,2046.0,1,1000200
...,...,...,...,...,...,...,...
17353,Paris,Charlie,MAGNIFICENT_MACARONS,SEASHELLS,745.0,5,3999600
17354,Charlie,Paris,SQUID_INK,SEASHELLS,1899.0,1,3999700
17355,Peter,Caesar,VOLCANIC_ROCK,SEASHELLS,10107.0,10,3999700
17356,Charlie,Paris,KELP,SEASHELLS,2011.0,1,3999700


In [6]:
mid_prices = price_df.pivot(index='t', columns='product', values='mid_price')
mid_prices

product,CROISSANTS,DJEMBES,JAMS,KELP,MAGNIFICENT_MACARONS,PICNIC_BASKET1,PICNIC_BASKET2,RAINFOREST_RESIN,SQUID_INK,VOLCANIC_ROCK,VOLCANIC_ROCK_VOUCHER_10000,VOLCANIC_ROCK_VOUCHER_10250,VOLCANIC_ROCK_VOUCHER_10500,VOLCANIC_ROCK_VOUCHER_9500,VOLCANIC_ROCK_VOUCHER_9750
t,,,,,,,,,,,,,,,
1000000,4265.5,13419.5,6519.5,2045.5,657.5,58707.0,30096.5,10000.0,1800.5,10218.5,233.5,63.5,8.5,718.5,469.5
1000100,4265.5,13419.5,6520.0,2045.0,661.5,58712.0,30097.5,9995.5,1801.0,10217.5,234.5,62.5,8.5,718.0,469.0
1000200,4265.5,13419.5,6519.5,2046.5,664.5,58715.5,30094.5,10000.0,1803.5,10222.0,235.5,64.5,9.5,722.5,473.5
1000300,4265.5,13419.5,6519.5,2045.5,669.5,58716.5,30095.5,10000.0,1802.5,10223.0,235.5,65.5,9.5,723.5,474.5
1000400,4266.5,13419.5,6519.5,2045.5,674.5,58715.5,30099.5,10000.0,1801.5,10217.0,234.5,62.5,8.5,717.5,468.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999500,4271.5,13409.5,6507.0,2013.0,745.5,58432.5,30384.5,10000.0,1906.5,10105.0,120.5,10.5,1.0,605.5,356.5
3999600,4271.0,13409.5,6508.0,2012.0,741.5,58435.0,30385.5,10004.5,1902.0,10106.0,122.5,10.5,1.0,606.5,356.5
3999700,4271.0,13409.5,6508.0,2013.0,745.5,58438.0,30385.0,10002.5,1900.5,10106.5,123.5,10.5,1.0,606.5,357.5


In [7]:
buyers = counterparty_df['buyer'].unique()
buyers

array(['Caesar', 'Charlie', 'Paris', 'Camilla', 'Pablo', 'Penelope',
       'Gary', 'Peter', 'Gina', 'Olivia'], dtype=object)

In [8]:
sellers = counterparty_df['seller'].unique()
sellers

array(['Paris', 'Caesar', 'Charlie', 'Gina', 'Gary', 'Pablo', 'Camilla',
       'Peter', 'Penelope', 'Olivia', 'Olga'], dtype=object)

In [9]:
for buyer in buyers:
    print(f'{buyer}:')
    print(counterparty_df.query('buyer == @buyer')['symbol'].value_counts())
    print()


Caesar:
symbol
JAMS                           2271
CROISSANTS                     1845
DJEMBES                        1693
VOLCANIC_ROCK                  1588
VOLCANIC_ROCK_VOUCHER_9750      692
VOLCANIC_ROCK_VOUCHER_9500      683
VOLCANIC_ROCK_VOUCHER_10000     656
VOLCANIC_ROCK_VOUCHER_10250     565
PICNIC_BASKET1                  529
SQUID_INK                       400
KELP                            397
MAGNIFICENT_MACARONS            387
VOLCANIC_ROCK_VOUCHER_10500     372
PICNIC_BASKET2                  290
RAINFOREST_RESIN                230
Name: count, dtype: int64

Charlie:
symbol
SQUID_INK               3854
KELP                    3773
RAINFOREST_RESIN        2202
PICNIC_BASKET2           238
MAGNIFICENT_MACARONS      39
PICNIC_BASKET1            17
Name: count, dtype: int64

Paris:
symbol
KELP                    2503
SQUID_INK               2454
DJEMBES                 1417
RAINFOREST_RESIN        1410
JAMS                    1242
CROISSANTS              1098
MAGNIFICENT_M

In [10]:
for seller in sellers:
    print(f'{seller}:')
    print(counterparty_df.query('seller == @seller')['symbol'].value_counts())
    print()


Paris:
symbol
KELP                    3244
SQUID_INK               3244
RAINFOREST_RESIN        1398
DJEMBES                 1366
JAMS                    1276
CROISSANTS              1174
MAGNIFICENT_MACARONS     433
Name: count, dtype: int64

Caesar:
symbol
VOLCANIC_ROCK_VOUCHER_9750     2172
VOLCANIC_ROCK_VOUCHER_10000    2162
VOLCANIC_ROCK_VOUCHER_9500     2115
VOLCANIC_ROCK_VOUCHER_10250    1966
VOLCANIC_ROCK                  1542
DJEMBES                        1416
JAMS                           1217
VOLCANIC_ROCK_VOUCHER_10500    1207
CROISSANTS                     1101
KELP                            521
SQUID_INK                       507
PICNIC_BASKET1                  414
PICNIC_BASKET2                  317
RAINFOREST_RESIN                246
MAGNIFICENT_MACARONS             28
Name: count, dtype: int64

Charlie:
symbol
KELP                    2275
SQUID_INK               2219
RAINFOREST_RESIN        2217
MAGNIFICENT_MACARONS     360
PICNIC_BASKET2           103
PICNIC_BASKET

In [11]:
counterparty_df.sort_values('t')

,buyer,seller,symbol,currency,price,quantity,t
35229,Caesar,Paris,CROISSANTS,SEASHELLS,4265.0,8,1000000
35230,Charlie,Paris,RAINFOREST_RESIN,SEASHELLS,9999.0,1,1000100
35231,Paris,Caesar,CROISSANTS,SEASHELLS,4266.0,8,1000200
35232,Paris,Caesar,JAMS,SEASHELLS,6520.0,7,1000200
35233,Paris,Charlie,KELP,SEASHELLS,2046.0,1,1000200
...,...,...,...,...,...,...,...
17352,Caesar,Pablo,VOLCANIC_ROCK,SEASHELLS,10104.0,2,3999600
17355,Peter,Caesar,VOLCANIC_ROCK,SEASHELLS,10107.0,10,3999700
17356,Charlie,Paris,KELP,SEASHELLS,2011.0,1,3999700
17354,Charlie,Paris,SQUID_INK,SEASHELLS,1899.0,1,3999700


In [12]:
def calculate_pnl(counterparty_df: pd.DataFrame, price_df: pd.DataFrame, party: str) -> float:
    pnl = 0
    holdings = {}
    for _, row in counterparty_df.iterrows():
        if row['buyer'] == party:
            holdings[row['symbol']] = holdings.get(row['symbol'], 0) + row['quantity']
            pnl -= row['quantity'] * row['price']
        elif row['seller'] == party:
            holdings[row['symbol']] = holdings.get(row['symbol'], 0) - row['quantity']
            pnl += row['quantity'] * row['price']
    for holding in holdings:
        pnl += holdings[holding] * price_df[(price_df['product'] == holding) & (price_df['t'] == counterparty_df['t'].max())]['mid_price'].item()
    return pnl

for party in sellers: # sellers has one more party than buyers
    print(f'{party}: {calculate_pnl(counterparty_df, price_df, party)}')

Paris: -76548.0
Caesar: -66625.0
Charlie: 242288.5
Gina: -8941.0
Gary: -50074.0
Pablo: -691450.0
Camilla: 1161727.5
Peter: -30078.0
Penelope: -109592.5
Olivia: 9152.0
Olga: 288.0


In [13]:
def calculate_per_product_pnl(counterparty_df: pd.DataFrame, price_df: pd.DataFrame, party: str) -> float:
    pnls = {}
    holdings = {}
    for _, row in counterparty_df.iterrows():
        if row['buyer'] == party:
            holdings[row['symbol']] = holdings.get(row['symbol'], 0) + row['quantity']
            pnls[row['symbol']] = pnls.get(row['symbol'], 0) + row['quantity'] * row['price']
        elif row['seller'] == party:
            holdings[row['symbol']] = holdings.get(row['symbol'], 0) - row['quantity']
            pnls[row['symbol']] = pnls.get(row['symbol'], 0) - row['quantity'] * row['price']
    # for holding in holdings:
    #     pnls[holding] += holdings[holding] * price_df[(price_df['product'] == holding) & (price_df['t'] == counterparty_df['t'].max())]['mid_price'].item()
    return pd.Series(pnls, name=party)

for party in sellers: # sellers has one more party than buyers
    print(calculate_per_product_pnl(counterparty_df, price_df, party))

CROISSANTS             -2710178.0
RAINFOREST_RESIN         616184.0
JAMS                   -2247624.0
KELP                   -1262014.0
SQUID_INK              -1268782.0
MAGNIFICENT_MACARONS      48738.0
DJEMBES                  525845.0
Name: Paris, dtype: float64
CROISSANTS                     22411612.0
JAMS                           50848616.0
MAGNIFICENT_MACARONS            1344821.0
DJEMBES                        29121841.0
VOLCANIC_ROCK_VOUCHER_9750     -6488670.0
VOLCANIC_ROCK_VOUCHER_9500    -10970089.0
VOLCANIC_ROCK_VOUCHER_10500      -46085.0
VOLCANIC_ROCK_VOUCHER_10250     -484061.0
VOLCANIC_ROCK_VOUCHER_10000    -2474388.0
VOLCANIC_ROCK                  90606636.0
PICNIC_BASKET1                 13003355.0
KELP                           -1639791.0
SQUID_INK                      -1522154.0
PICNIC_BASKET2                  -515683.0
RAINFOREST_RESIN                  27406.0
Name: Caesar, dtype: float64
RAINFOREST_RESIN          569279.0
KELP                     7743742.0
SQUID

In [14]:
camilla_pnls = calculate_per_product_pnl(counterparty_df, price_df, 'Camilla')
camilla_pnls.sort_values(ascending=False)

PICNIC_BASKET1                 38867970.0
PICNIC_BASKET2                 25181997.0
VOLCANIC_ROCK_VOUCHER_9500     13744040.0
VOLCANIC_ROCK_VOUCHER_9750      8057657.0
VOLCANIC_ROCK_VOUCHER_10000     3069253.0
VOLCANIC_ROCK_VOUCHER_10250      596933.0
MAGNIFICENT_MACARONS             548190.0
RAINFOREST_RESIN                 237396.0
VOLCANIC_ROCK_VOUCHER_10500       56377.0
KELP                            -361128.0
SQUID_INK                       -459075.0
CROISSANTS                    -19700909.0
DJEMBES                       -29647686.0
JAMS                          -48600992.0
Name: Camilla, dtype: float64

In [15]:
import plotly.graph_objects as go
import pandas as pd

def plot_trades(counterparty_df: pd.DataFrame, price_df: pd.DataFrame, party: str, product: str):
    # Filter trades for this party and product
    buys = counterparty_df[(counterparty_df['buyer'] == party) & (counterparty_df['symbol'] == product)]
    sells = counterparty_df[(counterparty_df['seller'] == party) & (counterparty_df['symbol'] == product)]
    # Get price data for this product
    prices = price_df[price_df['product'] == product]

    fig = go.Figure()

    # Mid price line
    fig.add_trace(go.Scatter(
        x=prices['t'],
        y=prices['mid_price'],
        mode='lines',
        name='Mid Price',
        line=dict(color='blue')
    ))

    # Entry (Buy) dots with quantity on hover
    fig.add_trace(go.Scatter(
        x=buys['t'],
        y=buys['price'],
        mode='markers',
        name='Entry (Buy)',
        marker=dict(color='green', size=8, symbol='circle'),
        customdata=buys['quantity'],
        hovertemplate='Time: %{x}<br>Price: %{y}<br>Quantity: %{customdata}<extra></extra>'
    ))

    # Exit (Sell) dots with quantity on hover
    fig.add_trace(go.Scatter(
        x=sells['t'],
        y=sells['price'],
        mode='markers',
        name='Exit (Sell)',
        marker=dict(color='red', size=8, symbol='circle'),
        customdata=sells['quantity'],
        hovertemplate='Time: %{x}<br>Price: %{y}<br>Quantity: %{customdata}<extra></extra>'
    ))

    fig.update_layout(
        title=f"{party}'s Trades for {product}",
        xaxis_title='t',
        yaxis_title='Price',
        legend=dict(x=0, y=1),
        template='plotly_white'
    )

    fig.show()

In [16]:
plot_trades(counterparty_df, price_df, 'Camilla', 'VOLCANIC_ROCK_VOUCHER_9500')

In [17]:
def calculate_per_product_pnl_over_time(
    counterparty_df: pd.DataFrame, 
    price_df: pd.DataFrame, 
    party: str
) -> pd.DataFrame:
    """
    Calculate per-product PnL for a party at every trade timestamp.

    Returns:
        pd.DataFrame: index is trade timestamp, columns are products, values are running PnL.
    """
    # Sort trades by time
    trades = counterparty_df.sort_values('t')
    products = counterparty_df['symbol'].unique()
    # Prepare mid price lookup: (product, t) -> mid_price
    price_lookup = price_df.set_index(['product', 't'])['mid_price'].to_dict()

    # Initialize
    holdings = {product: 0 for product in products}
    cash_pnl = {product: 0.0 for product in products}
    pnl_records = []

    for _, row in trades.iterrows():
        t = row['t']
        product = row['symbol']
        # Update holdings and cash PnL
        if row['buyer'] == party:
            holdings[product] += row['quantity']
            cash_pnl[product] -= row['quantity'] * row['price']
        elif row['seller'] == party:
            holdings[product] -= row['quantity']
            cash_pnl[product] += row['quantity'] * row['price']
        # Mark-to-market: PnL = cash + holdings * mid_price
        record = {'t': t}
        for prod in products:
            mid_price = price_lookup.get((prod, t), None)
            if mid_price is not None:
                record[prod] = cash_pnl[prod] + holdings[prod] * mid_price
            else:
                record[prod] = None
        pnl_records.append(record)

    pnl_df = pd.DataFrame(pnl_records).set_index('t')
    return pnl_df

calculate_per_product_pnl_over_time(counterparty_df, price_df, 'Camilla')

,CROISSANTS,RAINFOREST_RESIN,JAMS,KELP,SQUID_INK,PICNIC_BASKET1,PICNIC_BASKET2,MAGNIFICENT_MACARONS,DJEMBES,VOLCANIC_ROCK_VOUCHER_9750,VOLCANIC_ROCK_VOUCHER_9500,VOLCANIC_ROCK_VOUCHER_10500,VOLCANIC_ROCK_VOUCHER_10250,VOLCANIC_ROCK_VOUCHER_10000,VOLCANIC_ROCK
t,,,,,,,,,,,,,,,
1000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999600,-31111.0,2712.0,-176468.0,7016.0,-1209.0,49740.0,250666.5,37595.0,-161632.5,766074.5,876249.0,-40875.0,-352619.0,-7610.5,0.0
3999700,-31111.0,2664.0,-176468.0,6840.0,-846.0,51738.0,250248.0,40755.0,-161632.5,790825.5,876249.0,-40875.0,-352619.0,17382.5,0.0
3999700,-31111.0,2664.0,-176468.0,6840.0,-846.0,51738.0,250248.0,40755.0,-161632.5,790825.5,876249.0,-40875.0,-352619.0,17382.5,0.0


In [18]:
import plotly.graph_objects as go
import pandas as pd

def plot_trades_with_pnl(
    counterparty_df: pd.DataFrame,
    price_df: pd.DataFrame,
    pnl_over_time: pd.DataFrame,
    party: str,
    product: str
):
    # Filter trades for this party and product
    buys = counterparty_df[(counterparty_df['buyer'] == party) & (counterparty_df['symbol'] == product)]
    sells = counterparty_df[(counterparty_df['seller'] == party) & (counterparty_df['symbol'] == product)]
    prices = price_df[price_df['product'] == product]

    fig = go.Figure()

    # Mid price line (blue)
    fig.add_trace(go.Scatter(
        x=prices['t'],
        y=prices['mid_price'],
        mode='lines',
        name='Mid Price',
        line=dict(color='blue')
    ))

    # Entry (Buy) dots with quantity on hover (green)
    fig.add_trace(go.Scatter(
        x=buys['t'],
        y=buys['price'],
        mode='markers',
        name='Entry (Buy)',
        marker=dict(color='green', size=8, symbol='circle'),
        customdata=buys['quantity'],
        hovertemplate='Time: %{x}<br>Price: %{y}<br>Quantity: %{customdata}<extra></extra>'
    ))

    # Exit (Sell) dots with quantity on hover (red)
    fig.add_trace(go.Scatter(
        x=sells['t'],
        y=sells['price'],
        mode='markers',
        name='Exit (Sell)',
        marker=dict(color='red', size=8, symbol='circle'),
        customdata=sells['quantity'],
        hovertemplate='Time: %{x}<br>Price: %{y}<br>Quantity: %{customdata}<extra></extra>'
    ))

    # PnL over time (orange, secondary y-axis)
    if product in pnl_over_time.columns:
        fig.add_trace(go.Scatter(
            x=pnl_over_time.index,
            y=pnl_over_time[product],
            mode='lines',
            name='PnL (Mark-to-Market)',
            line=dict(color='orange', dash='dash'),
            yaxis='y2'
        ))

    # Layout with secondary y-axis for PnL
    fig.update_layout(
        title=f"{party}'s Trades and PnL for {product}",
        xaxis_title='t',
        yaxis=dict(
            title='Price',
            side='left'
        ),
        yaxis2=dict(
            title='PnL',
            overlaying='y',
            side='right',
            showgrid=False
        ),
        legend=dict(x=0, y=1),
        template='plotly_white'
    )

    fig.show()

In [19]:
plot_trades_with_pnl(counterparty_df, price_df, calculate_per_product_pnl_over_time(counterparty_df, price_df, 'Camilla'), 'Camilla', 'PICNIC_BASKET2')

In [20]:
price_df

,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss,round,t
0,PICNIC_BASKET2,30094.0,11.0,30093.0,27.0,NaN,NaN,30099.0,1.0,30100.0,37.0,NaN,NaN,30096.5,0.0,5,1000000
1,PICNIC_BASKET1,58702.0,11.0,58701.0,27.0,NaN,NaN,58712.0,11.0,58713.0,27.0,NaN,NaN,58707.0,0.0,5,1000000
2,VOLCANIC_ROCK_VOUCHER_10500,8.0,11.0,NaN,NaN,NaN,NaN,9.0,11.0,NaN,NaN,NaN,NaN,8.5,0.0,5,1000000
3,VOLCANIC_ROCK_VOUCHER_10000,233.0,11.0,NaN,NaN,NaN,NaN,234.0,11.0,NaN,NaN,NaN,NaN,233.5,0.0,5,1000000
4,JAMS,6519.0,59.0,6518.0,196.0,NaN,NaN,6520.0,59.0,6521.0,196.0,NaN,NaN,6519.5,0.0,5,1000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,PICNIC_BASKET1,58434.0,1.0,58433.0,10.0,58432.0,30.0,58443.0,1.0,58444.0,10.0,58445.0,30.0,58438.5,0.0,5,3999900
449996,VOLCANIC_ROCK_VOUCHER_9750,356.0,11.0,NaN,NaN,NaN,NaN,357.0,11.0,NaN,NaN,NaN,NaN,356.5,0.0,5,3999900
449997,RAINFOREST_RESIN,9992.0,35.0,NaN,NaN,NaN,NaN,9999.0,1.0,10008.0,35.0,NaN,NaN,9995.5,0.0,5,3999900
449998,VOLCANIC_ROCK_VOUCHER_10000,122.0,11.0,NaN,NaN,NaN,NaN,123.0,11.0,NaN,NaN,NaN,NaN,122.5,0.0,5,3999900


In [21]:
sellers

array(['Paris', 'Caesar', 'Charlie', 'Gina', 'Gary', 'Pablo', 'Camilla',
       'Peter', 'Penelope', 'Olivia', 'Olga'], dtype=object)

In [22]:
calculate_per_product_pnl(counterparty_df, price_df, 'Paris').sort_values(ascending=False)

RAINFOREST_RESIN         616184.0
DJEMBES                  525845.0
MAGNIFICENT_MACARONS      48738.0
KELP                   -1262014.0
SQUID_INK              -1268782.0
JAMS                   -2247624.0
CROISSANTS             -2710178.0
Name: Paris, dtype: float64

In [23]:
party, product = 'Caesar', 'MAGNIFICENT_MACARONS'
plot_trades_with_pnl(counterparty_df, price_df, calculate_per_product_pnl_over_time(counterparty_df, price_df, party), party, product)